In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from IPython.display import display
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind, chi2_contingency

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
input=pd.read_csv(r'C:\Users\User\Downloads/EEG.machinelearing_data_BRMH.csv')

In [ ]:
# Identify unnamed columns
unnamed_columns = [col for col in input.columns if 'Unnamed' in col]
if not unnamed_columns:
    print("No unnamed columns found.")
else:
    print("Unnamed columns:", unnamed_columns)
    print(" Dropping it.")

# Check if unnamed columns are empty
empty_unnamed_columns = [col for col in unnamed_columns if input[col].isnull().all()]

# Drop empty unnamed columns
input.drop(empty_unnamed_columns, axis=1, inplace=True)

unnamed_columns = [col for col in input.columns if 'Unnamed' in col]
if not unnamed_columns:
    print("No unnamed columns found.")
else:
    print("Unnamed columns:", unnamed_columns)

In [ ]:
input.head()

In [ ]:
input.info()

In [ ]:
columns_of_interest = input.columns  # Adjust column indices as needed

nan_counts_specific = input[columns_of_interest].isna().sum()
nan_counts_specific = nan_counts_specific[nan_counts_specific != 0]

if not nan_counts_specific.empty:
    print("NaN counts for specified columns:")
    print(nan_counts_specific)
else:
    print("No NaN values found in specified columns.")

In [ ]:
columns_of_interest = ['sex','education','IQ','main.disorder','specific.disorder']
# Count unique values in specified columns
unique_value_counts = input[columns_of_interest].nunique()

# Display unique value counts for specified columns
print("Unique value counts in specified columns:")
print(unique_value_counts)

In [ ]:
# Identify columns with NaN values
columns_with_nan = nan_counts_specific.index.tolist()

# Drop rows where any of the specified columns have NaN values
input.dropna(subset=columns_with_nan, inplace=True)

# Confirm that rows with NaN values in specified columns are dropped
print("After dropping rows with NaN values:")
print(input.shape)  # Check the shape of the DataFrame after dropping rows

columns_of_interest = input.columns  # Adjust column indices as needed

nan_counts_specific = input[columns_of_interest].isna().sum()
nan_counts_specific = nan_counts_specific[nan_counts_specific != 0]

if not nan_counts_specific.empty:
    print("NaN counts for specified columns:")
    print(nan_counts_specific)
else:
    print("No NaN values found in specified columns.")

In [ ]:
# Check the unique values in each column
for column in input.columns[1:8]:
    unique_values = sorted(input[column].unique())
    print(f"Column '{column}' has {len(unique_values)} unique values: {unique_values}")

In [11]:
input.drop(columns=['eeg.date'], inplace=True)

In [ ]:
input.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns and store mappings
label_encoders = {}
mappings = {}
columns=['sex','main.disorder','specific.disorder']
for column in columns:
    le = LabelEncoder()
    input[column] = le.fit_transform(input[column])
    label_encoders[column] = le
    mappings[column] = dict(zip(le.classes_, le.transform(le.classes_)))

# Check the unique values in each column and their mappings
for column in columns:
    unique_values = sorted(input[column].unique())
    print(f"Column '{column}' has {len(unique_values)} unique values: {unique_values}")
    print(f"Mappings: {mappings[column]}")

# Display the encoded data
#print(input)

In [ ]:
input.describe()

In [ ]:
input_copy=input.copy().iloc[:, np.r_[1:6]]

corr_matrix = input_copy.corr()
display(corr_matrix)

In [ ]:
plt.figure(figsize=(8, 4))  # Set the size of the plot
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix Heatmap')
plt.show()

In [17]:
from statsmodels.stats.multitest import multipletests
from scipy.stats import pearsonr
import networkx as nx

data=input_copy
#Calculate correlation matrix and p-values
correlations = data.corr()
p_values = np.zeros_like(correlations)

for i in range(data.shape[1]):
    for j in range(data.shape[1]):
        if i != j:
            _, p_values[i, j] = pearsonr(data.iloc[:, i], data.iloc[:, j])

# Apply Benjamini-Hochberg correction for multiple comparisons
p_values_flat = p_values.flatten()
_, corrected_p_values_flat, _, _ = multipletests(p_values_flat, method='fdr_bh')
corrected_p_values = corrected_p_values_flat.reshape(p_values.shape)

# Apply thresholds (r >= 0.25 and p < 0.05)
significant_edges = np.where((np.abs(correlations) >= 0.15) & (corrected_p_values < 0.05), correlations, 0)

In [ ]:
# Create an undirected graph
G = nx.Graph()

# Add nodes (symptoms)
for col in data.columns:
    G.add_node(col)

# Add edges for significant correlations with edge weights
for i in range(len(data.columns)):
    for j in range(i + 1, len(data.columns)):  # Avoid duplicate edges
        if significant_edges[i, j] != 0:
            # Determine edge color based on the sign of correlation
            color = 'green' if significant_edges[i, j] > 0 else 'red'
            G.add_edge(data.columns[i], data.columns[j], weight=significant_edges[i, j], color=color)

# Extract edges and colors
edges = G.edges(data=True)
weights = [d['weight'] for (u, v, d) in edges]
colors = [d['color'] for (u, v, d) in edges]

# Normalize weights for edge width
norm_weights = [abs(w) for w in weights]

# Position nodes with a spring layout
pos = nx.circular_layout(G)

# Plot the graph
plt.figure(figsize=(8, 8))

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=700)

# Draw edges with color coding based on weights
nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=colors, width=[w*7 for w in norm_weights])

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=10, font_color="black")

plt.title('Correlation Network with Weighted and Color-Coded Edges')

# Show the plot
plt.show()

In [ ]:
import networkx as nx
from networkx.algorithms.clique import find_cliques
from networkx.algorithms.approximation import clique

### Identifying Cliques

# Identify all maximal cliques
cliques = list(find_cliques(G))

# Find the largest maximal clique (i.e., the maximum clique)
maximum_clique = max(cliques, key=len)

print("Maximal Cliques: ", cliques)
print("Maximum Clique: ", maximum_clique)

### Visualize Cliques

# Create a new plot to visualize the cliques
plt.figure(figsize=(8, 8))

# Draw the base network again
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=700)
nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=colors, width=[w*5 for w in norm_weights])
nx.draw_networkx_labels(G, pos, font_size=10, font_color="black")

# Highlight maximal cliques (all cliques) with blue edges
for clique in cliques:
    clique_edges = [(clique[i], clique[j]) for i in range(len(clique)) for j in range(i + 1, len(clique))]
    nx.draw_networkx_edges(G, pos, edgelist=clique_edges, edge_color='blue', width=3, alpha=0.6)

# Highlight the maximum clique with darker blue edges
max_clique_edges = [(maximum_clique[i], maximum_clique[j]) for i in range(len(maximum_clique)) for j in range(i + 1, len(maximum_clique))]
nx.draw_networkx_edges(G, pos, edgelist=max_clique_edges, edge_color='darkblue', width=5, alpha=1.0)

plt.title('Correlation Network with Maximal and Maximum Cliques')

# Show the plot
plt.show()

In [ ]:
from sklearn.covariance import GraphicalLassoCV

# Fit a Graphical Lasso model to estimate partial correlations
model = GraphicalLassoCV()
model.fit(data)

# Extract the partial correlation matrix
partial_corr_matrix = -model.precision_ / np.sqrt(np.outer(np.diag(model.precision_), np.diag(model.precision_)))

# Create a graph for partial correlations
G_partial = nx.Graph()

# Add nodes (variables)
for col in data.columns:
    G_partial.add_node(col)

# Add edges (partial correlations) only if significant
for i, col1 in enumerate(data.columns):
    for j, col2 in enumerate(data.columns):
        if i != j and abs(partial_corr_matrix[i, j]) >= 0.25:
            G_partial.add_edge(col1, col2, weight=partial_corr_matrix[i, j])

# Plot the partial correlation network
plt.figure(figsize=(8, 8))
pos = nx.circular_layout(G_partial)
edges = G_partial.edges(data=True)

nx.draw(G_partial, pos, with_labels=True, node_size=500, node_color='lightgreen', font_size=10)
nx.draw_networkx_edges(G_partial, pos, edgelist=edges, width=[abs(d['weight'])*5 for (u, v, d) in edges])

plt.title('Partial Correlation Network')
plt.show()


In [ ]:
# Function to create pie charts
def create_pie_chart(ax, column_name, data, mappings):
    counts = data[column_name].value_counts()
    labels = [list(mappings[column_name].keys())[list(mappings[column_name].values()).index(i)] for i in counts.index]
    ax.pie(counts, labels=labels, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
    ax.set_title(f'Distribution of {column_name}')
    ax.axis('equal')

#columns we want to work on
columns_to_plot = ['sex', 'main.disorder', 'specific.disorder']
# Number of columns for the subplot grid
num_columns = 3
num_plots = len(columns_to_plot)

# Calculate number of rows needed
num_rows = (num_plots + num_columns - 1) // num_columns

fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 5))

# Flatten axes array if there's more than one row
if num_rows > 1:
    axes = axes.flatten()

# Create pie charts for each categorical variable
for i, column in enumerate(columns_to_plot):
    create_pie_chart(axes[i], column, input, mappings)

# Remove any unused subplots
for i in range(num_plots, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [ ]:
# Decoding the columns back to original values
decoded_data = input.copy()
for column in columns:
    le = label_encoders[column]
    decoded_data[column] = input[column].map({v: k for k, v in mappings[column].items()})

# Group by 'main.disorder' and list unique 'specific.disorder' for each group
main_to_specific = decoded_data.groupby('main.disorder')['specific.disorder'].unique()

# Convert the result to a dictionary for easier inspection
main_to_specific_dict = main_to_specific.to_dict()

print(main_to_specific_dict)

In [ ]:
# Create mappings from your label encoders (example mappings provided)
mappings = {
    'main.disorder': {
        'Addictive disorder': 0, 'Anxiety disorder': 1, 'Healthy control': 2,
        'Mood disorder': 3, 'Obsessive compulsive disorder': 4, 'Schizophrenia': 5,
        'Trauma and stress related disorder': 6
    },
    'specific.disorder': {
        'Acute stress disorder': 0, 'Adjustment disorder': 1, 'Alcohol use disorder': 2,
        'Behavioral addiction disorder': 3, 'Bipolar disorder': 4, 'Depressive disorder': 5,
        'Healthy control': 6, 'Obsessive compulsitve disorder': 7, 'Panic disorder': 8,
        'Posttraumatic stress disorder': 9, 'Schizophrenia': 10, 'Social anxiety disorder': 11
    }
}

# Group data by main disorder
groups = {}
for main_disorder, specific_disorders in main_to_specific.items():
    group_data = input[input['main.disorder'] == mappings['main.disorder'][main_disorder]]
    for specific_disorder in specific_disorders:
        group_data_specific = group_data[group_data['specific.disorder'] == mappings['specific.disorder'][specific_disorder]]
        groups[specific_disorder] = group_data_specific

# Perform t-tests for continuous variables
continuous_vars = ['age', 'education', 'IQ']
t_test_results = {}

for var in continuous_vars:
    for specific_disorder, group_data in groups.items():
        hc_group = groups.get('Healthy control')
        if hc_group is not None:
            hc_values = hc_group[var].dropna()
            group_values = group_data[var].dropna()
            if hc_values.empty or group_values.empty:
                print(f"Skipping t-test for {specific_disorder} on {var} due to insufficient data.")
                t_test_results[(specific_disorder, var)] = {'t_stat': np.nan, 'p_value': np.nan}
            else:
                t_stat, p_value = ttest_ind(group_values, hc_values)
                t_test_results[(specific_disorder, var)] = {'t_stat': t_stat, 'p_value': p_value}
        else:
            print(f"Healthy control group not found for {specific_disorder} on {var}.")

print("T-test Results:")
print(t_test_results)

# Perform chi-squared tests for categorical variables
categorical_vars = ['sex']
chi2_results = {}

for var in categorical_vars:
    for specific_disorder, group_data in groups.items():
        hc_group = groups.get('Healthy control')
        if hc_group is not None:
            contingency_table = pd.crosstab(group_data[var], hc_group[var])
            if contingency_table.size == 0:
                print(f"Skipping chi-squared test for {specific_disorder} on {var} due to insufficient data.")
                chi2_results[(specific_disorder, var)] = {'chi2': np.nan, 'p_value': np.nan}
            else:
                chi2, p_value, dof, expected = chi2_contingency(contingency_table)
                chi2_results[(specific_disorder, var)] = {'chi2': chi2, 'p_value': p_value}
        else:
            print(f"Healthy control group not found for {specific_disorder} on {var}.")

print("Chi-squared Test Results:")
print(chi2_results)



def chi_square_test(column1, column2, data):
    contingency_table = pd.crosstab(data[column1], data[column2])
    chi2, p, dof, ex = chi2_contingency(contingency_table)
    return chi2, p

chi_square_results = {}
categorical_columns = ['main.disorder', 'specific.disorder']

# Iterate through pairs of columns and their unique values
for column2 in categorical_columns:
    unique_values = input[column2].unique()
    for unique_value in unique_values:
        # Create a new binary column for the unique value
        input[f'{column2}_{unique_value}'] = (input[column2] == unique_value).astype(int)

        # Perform chi-square test between 'sex' and the new binary column
        chi2, p_value = chi_square_test('sex', f'{column2}_{unique_value}', input)
        chi_square_results[('sex', f'{column2}_{unique_value}')] = (chi2, p_value)

# Print the chi-square statistics and p-values for each pair of variables
for key, value in chi_square_results.items():
    chi2, p_value = value

    print(f"Chi-square test between {key[0]} and {key[1]}: chi2 = {chi2}, p-value = {p_value}")

In [ ]:
print(input.columns)

In [ ]:
# Assuming input is your DataFrame
# Create mappings from your label encoders (example mappings provided)
mappings = {
    'main.disorder': {
        'Addictive disorder': 0, 'Anxiety disorder': 1, 'Healthy control': 2,
        'Mood disorder': 3, 'Obsessive compulsive disorder': 4, 'Schizophrenia': 5,
        'Trauma and stress related disorder': 6
    },
    'specific.disorder': {
        'Acute stress disorder': 0, 'Adjustment disorder': 1, 'Alcohol use disorder': 2,
        'Behavioral addiction disorder': 3, 'Bipolar disorder': 4, 'Depressive disorder': 5,
        'Healthy control': 6, 'Obsessive compulsitve disorder': 7, 'Panic disorder': 8,
        'Posttraumatic stress disorder': 9, 'Schizophrenia': 10, 'Social anxiety disorder': 11
    }
}

# Example mapping from main disorder to specific disorders
main_to_specific = {
    'Addictive disorder': ['Alcohol use disorder', 'Behavioral addiction disorder'],
    'Anxiety disorder': ['Panic disorder', 'Social anxiety disorder'],
    'Healthy control': ['Healthy control'],
    'Mood disorder': ['Depressive disorder', 'Bipolar disorder'],
    'Obsessive compulsive disorder': ['Obsessive compulsitve disorder'],
    'Schizophrenia': ['Schizophrenia'],
    'Trauma and stress related disorder': ['Acute stress disorder', 'Posttraumatic stress disorder', 'Adjustment disorder']
}

# Group data by main disorder
groups = {}
for main_disorder, specific_disorders in main_to_specific.items():
    group_data = input[input['main.disorder'] == mappings['main.disorder'][main_disorder]]
    for specific_disorder in specific_disorders:
        group_data_specific = group_data[group_data['specific.disorder'] == mappings['specific.disorder'][specific_disorder]]
        groups[specific_disorder] = group_data_specific

# Perform t-tests for continuous variables
continuous_vars = ['age', 'education', 'IQ']
t_test_results = {}

for var in continuous_vars:
    for specific_disorder, group_data in groups.items():
        hc_group = groups.get('Healthy control')
        if hc_group is not None:
            hc_values = hc_group[var].dropna()
            group_values = group_data[var].dropna()

            if hc_values.empty or group_values.empty:
                print(f"Skipping t-test for {specific_disorder} on {var} due to insufficient data.")
                t_test_results[(specific_disorder, var)] = {'t_stat': np.nan, 'p_value': np.nan}
            else:
                mean_hc = np.mean(group_values)
                std_hc = np.std(group_values)
                t_stat, p_value = ttest_ind(group_values, hc_values)
                t_test_results[(specific_disorder, var)] = {'t_stat': t_stat, 'p_value': p_value, 'mean': mean_hc, 'SD': std_hc}
        else:
            print(f"Healthy control group not found for {specific_disorder} on {var}.")

print("T-test Results:")
print(t_test_results)

# Perform chi-squared tests for categorical variables
categorical_vars = ['sex']
chi2_results = {}

for var in categorical_vars:
    for specific_disorder, group_data in groups.items():
        hc_group = groups.get('Healthy control')
        if hc_group is not None:
            if var in group_data.columns and var in hc_group.columns:
                group_counts = group_data[var].value_counts()
                hc_counts = hc_group[var].value_counts()
                contingency_table = pd.DataFrame([group_counts, hc_counts]).fillna(0)
                print(f"Contingency table for {specific_disorder} on {var}:\n{contingency_table}")
                if contingency_table.size == 0 or (contingency_table == 0).all().all():
                    print(f"Skipping chi-squared test for {specific_disorder} on {var} due to insufficient data.")
                    chi2_results[(specific_disorder, var)] = {'chi2': np.nan, 'p_value': np.nan}
                else:
                    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
                    chi2_results[(specific_disorder, var)] = {'chi2': chi2, 'p_value': p_value}
            else:
                print(f"{var} not found in group data or healthy control data for {specific_disorder}.")
                chi2_results[(specific_disorder, var)] = {'chi2': np.nan, 'p_value': np.nan}
        else:
            print(f"Healthy control group not found for {specific_disorder} on {var}.")

print("Chi-squared Test Results:")
print(chi2_results)



In [ ]:
# Create a DataFrame to store the results
results = []

# Populate the results DataFrame
for main_disorder, specific_disorders in main_to_specific.items():
    for specific_disorder in specific_disorders:
        row = {'Main/specific  ': specific_disorder}

        # Age statistics
        age_stats = t_test_results.get((specific_disorder, 'age'), {})
        row['  Age Mean(SD)'] = f"{age_stats.get('mean', np.nan):.2f}({age_stats.get('SD', np.nan):.2f})"
        row['  Age t'] = f"{age_stats.get('t_stat', np.nan):.2f}"

        # Sex statistics
        sex_counts = groups[specific_disorder]['sex'].value_counts(normalize=True).fillna(0) * 100
        male_count = sex_counts.get(1, 0)
        female_count = sex_counts.get(0, 0)
        sex_chi2 = chi2_results.get((specific_disorder, 'sex'), {}).get('chi2', np.nan)
        row['       Sex Counts(proportions)'] = f"Male: {male_count:.1f}%, Female: {female_count:.1f}%"
        row['  Sex χ2'] = f"{sex_chi2:.2f}"

        # Education statistics
        education_stats = t_test_results.get((specific_disorder, 'education'), {})
        row['  Education Mean(SD)'] = f"{education_stats.get('mean', np.nan):.2f}({education_stats.get('SD', np.nan):.2f})"
        row['  Education t'] = f"{education_stats.get('t_stat', np.nan):.2f}"

        # IQ statistics
        iq_mean = groups[specific_disorder]['IQ'].mean()
        row['  IQ Mean(SD)'] = f"{iq_mean:.2f}"

        results.append(row)

results_df = pd.DataFrame(results)

# Display the results DataFrame in a single line for each row
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
print(results_df.to_string(index=False))

In [ ]:
input.head()

### TABLE 1 | Demographic characteristics of samples(ORIGINAL PAPER)

| Main/specific              | Age            | Sex                                 | Education       | IQ                |
|----------------------------|----------------|-------------------------------------|-----------------|-------------------|
|                            | Mean(SD)       | Counts(proportions)                 | Mean(SD)        | Mean(SD)          |
|----------------------------|----------------|-------------------------------------|-----------------|-------------------|
| Healthy control (n=95)     | 25.72(4.55)    | Male:60(63.2%) Female:35(36.8%)     | 14.91(2.06)     | 116.24(10.94)     |
| Schizophrenia (n=117)      | 31.73(12.10)   | 4.58*** Male:65(55.6%) Female:52(44.4%) | 1.25 12.84(2.95) | −5.76*** 89.62(17.51) |
| Mood disorder (n=266)      | 30.87(12.70)   | 3.86*** Male:151(56.8%) Female:115(43.2%) | 1.17 13.31(2.48) | −5.59*** 101.58(15.70) |
| Depressive disorder (n=199)| 31.26(13.23)   | 3.96*** Male:109(54.8%) Female:90(45.2%) | 1.84 13.05(2.51) | −6.25*** 101.85(15.28) |
| Bipolar disorder (n=67)    | 29.71(11.01)   | 3.17** Male:42(62.7%) Female:25(37.3%) | 0.00 14.11(2.21) | −2.36* 100.81(16.98) |
| Anxiety disorder (n=107)   | 29.01(10.56)   | 2.81** Male:79(73.8%) Female:28(26.2%) | 2.67 13.14(2.42) | −5.52*** 98.31(16.31) |
| Panic disorder (n=59)      | 31.05(11.30)   | 4.10*** Male:38(64.4%) Female:21(35.6%) | 0.25 13.45(2.91) | −3.62*** 100.31(14.77) |
| Social anxiety disorder (n=48)| 26.51(9.09) | 0.69 Male:41(85.4%) Female:7(14.6%) | 7.61** 12.78(1.60) | −6.28*** 95.85(17.89) |
| Obsessive–compulsive disorder (n=46)| 28.48(9.83) | 2.28* Male:38(82.6%) Female:8(17.4%) | 5.53* 13.93(2.33) | −2.45* 107.80(15.24) |
| Addictive disorder (n=186) | 29.63(10.89)   | 3.34*** Male:164(88.2%) Female:22(11.8%) | 24.33*** 13.23(2.53) | −5.55*** 103.88(16.19) |
| Alcohol use disorder (n=93)| 34.16(11.88)   | 6.45*** Male:75(80.6%) Female:18(19.4%) | 7.09** 13.29(3.07) | −4.22*** 103.38(13.61) |
| Behavioral addiction disorder (n=93)| 25.09(7.48) | −0.70 Male:89(95.7%) Female:4(4.3%) | 30.26*** 13.16(1.89) | −6.02*** 104.38(18.49) |
| Trauma and stress-related disorder (n=128)| 36.09(13.82) | 7.03*** Male:44(34.4%) Female:84(65.6%) | 18.15*** 13.57(2.45) | −4.28*** 98.89(15.86) |
| Post-traumatic stress disorder (n=52)| 42.74(13.0) | 11.55*** Male:14(26.9%) Female:38(73.1%) | 17.65*** 13.37(2.54) | −3.95*** 98.90(15.69) |
| Acute stress disorder (n=38)| 28.90(9.05) | 2.69** Male:3(7.9%) Female:35(92.1%) | 33.25*** 14.26(2.27) | −1.59 104.06(15.43) |
| Adjustment disorder (n=38)| 34.19(14.90) | 5.01*** Male:27(75.0%) Female:11(25.0%) | 0.74 13.26(2.41) | −4.21*** 94.24(15.41) |


In [ ]:
print(input.columns[8:1148])

# **Correlation Network Analysis**

In [ ]:
corr_matrix = input.iloc[:,np.r_[2:7]].corr()
display(corr_matrix)
plt.figure(figsize=(8, 4))  # Set the size of the plot
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix Heatmap')
plt.show()

In [68]:
from statsmodels.stats.multitest import multipletests
from scipy.stats import pearsonr
import networkx as nx

data=input.copy()
data = data.iloc[:, np.r_[2:6]]
#Calculate correlation matrix and p-values
correlations = data.corr()
p_values = np.zeros_like(correlations)

for i in range(data.shape[1]):
    for j in range(data.shape[1]):
        if i != j:
            _, p_values[i, j] = pearsonr(data.iloc[:, i], data.iloc[:, j])

# Apply Benjamini-Hochberg correction for multiple comparisons
p_values_flat = p_values.flatten()
_, corrected_p_values_flat, _, _ = multipletests(p_values_flat, method='fdr_bh')
corrected_p_values = corrected_p_values_flat.reshape(p_values.shape)

# Apply thresholds (r >= 0.15 and p < 0.05)
significant_edges = np.where((np.abs(correlations) >= 0.15) & (corrected_p_values < 0.05), correlations, 0)

In [ ]:
# Create an undirected graph
G = nx.Graph()

# Add nodes (symptoms)
for col in data.columns:
    G.add_node(col)


# Add edges for significant correlations with edge weights
for i in range(len(data.columns)):
    for j in range(i + 1, len(data.columns)):  # Avoid duplicate edges
        if significant_edges[i, j] != 0:
            # Determine edge color based on the sign of correlation
            color = 'green' if significant_edges[i, j] > 0 else 'red'
            G.add_edge(data.columns[i], data.columns[j], weight=significant_edges[i, j], color=color)

# Extract edges and colors
edges = G.edges(data=True)
weights = [d['weight'] for (u, v, d) in edges]
colors = [d['color'] for (u, v, d) in edges]


# Normalize weights for edge width
norm_weights = [abs(w) for w in weights]

# ciruclar layout for easy visualization
pos = nx.circular_layout(G)

# Plot the graph
plt.figure(figsize=(8, 8))

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=700)

# Draw edges with color coding based on weights
nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=colors, width=[w*7 for w in norm_weights])

# Draw labels
nx.draw_networkx_labels(G, pos,font_size=10, font_color="black")

plt.title('Correlation Network with Weighted and Color-Coded Edges')

# Show the plot
plt.show()

In [ ]:
# Original node names
node_names = {
    'age': 'Age',
    'main.disorder': 'Main\nDisorder',
    'education': 'Education',
    'IQ': 'IQ'
}



# Rename nodes in the graph
G = nx.relabel_nodes(G, node_names)

# Extract edges and colors again after renaming nodes
edges = G.edges(data=True)
weights = [d['weight'] for (u, v, d) in edges]
colors = [d['color'] for (u, v, d) in edges]

# Normalize weights for edge width
norm_weights = [abs(w) for w in weights]

# Circular layout for easy visualization
pos = nx.circular_layout(G)

# Plot the graph
plt.figure(figsize=(7, 5.7))

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_color='white', node_size=3100,edgecolors='black')

# Draw edges with color coding based on weights
nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=colors, width=[w*10 for w in norm_weights])

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=10, font_color="black")

plt.axis('off')

#Save the figure
plt.savefig('content/Correlation Network with Weighted and Color-Coded Edges-3.png', dpi=500, bbox_inches='tight')
plt.savefig('content/Correlation Network with Weighted and Color-Coded Edges-3.pdf', dpi=500, bbox_inches='tight')
# Show the plot
plt.show()


In [ ]:
# Identify all maximal cliques
cliques = list(find_cliques(G))

# Find the largest maximal clique (i.e., the maximum clique)
maximum_clique = max(cliques, key=len)

print("Maximal Cliques: ", cliques)
print("Maximum Clique: ", maximum_clique)

In [ ]:
# Calculate correlation matrix
correlations = data.corr()

# Calculate p-values for each correlation
p_values = np.zeros_like(correlations)
for i in range(data.shape[1]):
    for j in range(data.shape[1]):
        if i != j:
            _, p_values[i, j] = pearsonr(data.iloc[:, i], data.iloc[:, j])

# Apply Benjamini-Hochberg correction
p_values_flat = p_values.flatten()
_, corrected_p_values_flat, _, _ = multipletests(p_values_flat, method='fdr_bh')
corrected_p_values = corrected_p_values_flat.reshape(p_values.shape)

# Create a matrix for storing the correlation coefficients and significance asterisks
formatted_matrix = pd.DataFrame(index=correlations.index, columns=correlations.columns)

# Apply thresholds and add stars based on significance levels
for i in range(data.shape[1]):
    for j in range(i + 1):
        corr_value = correlations.iloc[i, j]
        if corrected_p_values[i, j] < 0.001:
            star = '***'
        elif corrected_p_values[i, j] < 0.01:
            star = '**'
        elif corrected_p_values[i, j] < 0.05:
            star = '*'
        else:
            star = ''
        formatted_matrix.iloc[i, j] = f"{corr_value:.2f}{star}"
        formatted_matrix.iloc[j, i] = ''  # Set upper triangle to empty

# Convert the formatted matrix to a LaTeX table format
latex_table = formatted_matrix.to_latex()

# Display the formatted correlation matrix
print("Formatted Correlation Matrix:\n")
print(formatted_matrix)

# Display the LaTeX table
print("\nLaTeX Table:\n")
print(latex_table)

In [35]:
# import networkx as nx
# import matplotlib.pyplot as plt
# from scipy.stats import pearsonr
# from statsmodels.stats.multitest import multipletests
# import numpy as np

# # Define thresholds
# r_threshold = 0.15
# p_threshold = 0.08

# # Define disorder mappings
# disorder_mappings = {
#     'Addictive disorder': 0,
#     'Anxiety disorder': 1,
#     'Healthy control': 2,
#     'Mood disorder': 3,
#     'Obsessive compulsive disorder': 4,
#     'Schizophrenia': 5,
#     'Trauma and stress related disorder': 6
# }

# # Loop through each disorder and calculate the correlation network
# for disorder_name, disorder_code in disorder_mappings.items():
#     # Filter the dataset for the current disorder
#     filtered_data = input_copy[input_copy['main.disorder'] == disorder_code]
#     filtered_data = filtered_data.drop(columns=['main.disorder'])

#     # Calculate the correlation matrix and p-values
#     correlations = filtered_data.corr()
#     p_values = np.zeros_like(correlations)

#     for i in range(filtered_data.shape[1]):
#         for j in range(filtered_data.shape[1]):
#             if i != j:
#                 _, p_values[i, j] = pearsonr(filtered_data.iloc[:, i], filtered_data.iloc[:, j])

#     # Apply Benjamini-Hochberg correction
#     p_values_flat = p_values.flatten()
#     _, corrected_p_values_flat, _, _ = multipletests(p_values_flat, method='fdr_bh')
#     corrected_p_values = corrected_p_values_flat.reshape(p_values.shape)

#     # Threshold for significant correlations
#     significant_edges = np.where((np.abs(correlations) >= r_threshold) & (corrected_p_values < p_threshold), correlations, 0)

#     # Build the network
#     G = nx.Graph()
#     for i in range(significant_edges.shape[0]):
#         for j in range(i+1, significant_edges.shape[1]):
#             if significant_edges[i, j] != 0:
#                 G.add_edge(filtered_data.columns[i], filtered_data.columns[j], weight=significant_edges[i, j])

#     # Identify cliques
#     cliques = list(nx.find_cliques(G))
#     maximum_clique = max(cliques, key=len)

#     # Visualize the network
#     plt.figure(figsize=(8, 8))
#     pos = nx.circular_layout(G)
#     edges = G.edges(data=True)
#     weights = [d['weight'] for (u, v, d) in edges]
#     norm_weights = [abs(w) for w in weights]

#     nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=700)
#     nx.draw_networkx_edges(G, pos, edgelist=edges, width=[w*7 for w in norm_weights])
#     nx.draw_networkx_labels(G, pos, font_size=10, font_color="black")

#     plt.title(f'Correlation Network for {disorder_name}')
#     plt.axis('off')
#     plt.show()

#     print(f"Maximal Cliques for {disorder_name}: ", cliques)
#     print(f"Maximum Clique for {disorder_name}: ", maximum_clique)
#     print("\n" + "="*50 + "\n")


# **EEG Related Analysis**

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Select EEG features
eeg_features = input.columns[7:1148]  # Assuming EEG features start from the 11th column
scaler = StandardScaler()
#eeg_features = input[eeg_features].select_dtypes(include=[float, int]).columns
scaled_data = scaler.fit_transform(input[eeg_features])

# Perform PCA
pca = PCA(n_components=10)
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame with principal components
pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(10)])
pca_df['Main Disorder'] = input['main.disorder']
print(pca.explained_variance_ratio_)

# Plot the principal components
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Main Disorder', data=pca_df, palette='viridis')
plt.title('PCA of EEG Data')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Main Disorder')
plt.show()

In [ ]:
import numpy as np
from scipy.signal import welch
import matplotlib.pyplot as plt

# Define a function to calculate PSD
def calculate_psd(data, fs=250):
    freqs, psd = welch(data, fs=fs, nperseg=1024)
    return freqs, psd

# Plot PSD for different disorders
plt.figure(figsize=(12, 8))

for disorder in input['main.disorder'].unique():
    disorder_indices = input['main.disorder'] == disorder
    disorder_data = input.loc[disorder_indices, input.columns[7:1148]].values.T  # Transpose for channel-wise PSD

    # Calculate average PSD across all channels for this disorder
    psd_list = []
    for channel_data in disorder_data:
        freqs, psd = calculate_psd(channel_data)
        psd_list.append(psd)

    mean_psd = np.mean(psd_list, axis=0)
    plt.plot(freqs, mean_psd, label=disorder)

plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density')
plt.title('PSD of EEG Data for Different Disorders')
plt.legend(title='Main Disorder')
plt.show()


In [ ]:
# Function to perform PCA and create a DataFrame with principal components
def perform_pca(data, n_components=2):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(scaled_data)
    return pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(n_components)])

# Group data by a demographic factor, e.g., gender
for gender in input['sex'].unique():
    gender_data = input[input['sex'] == gender]
    gender_eeg_data = gender_data.iloc[:, 7:1148].values

    # Perform PCA on the subgroup
    pca_df = perform_pca(gender_eeg_data)
    pca_df['Main_Disorder'] = gender_data['main.disorder'].values  # Ensure this is added correctly
    pca_df['sex'] = gender  # Add the sex information to the DataFrame

    # Plot PCA for the subgroup
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='PC1', y='PC2', hue='Main_Disorder', data=pca_df, palette='viridis')
    plt.title(f'PCA of EEG Data for Gender {gender}')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend(title='Main Disorder')
    plt.show()

# Combine all PCA results for the box plot
combined_pca_df = pd.concat([perform_pca(input[input['sex'] == gender].iloc[:, 7:1148].values).assign(Main_Disorder=input[input['sex'] == gender]['main.disorder'].values, sex=gender) for gender in input['sex'].unique()])

# Box plot to compare EEG feature (e.g., a specific principal component) across subgroups
plt.figure(figsize=(12, 8))
sns.boxplot(x='Main_Disorder', y='PC1', hue='sex', data=combined_pca_df)
plt.title('Comparison of PC1 across Disorders and Gender')
plt.xlabel('Main Disorder')
plt.ylabel('Principal Component 1')
plt.show()

In [ ]:
plt.rc('font', family='serif', size=11)
# Assuming df is your DataFrame containing the dataset
plt.figure(figsize=(12, 6))
sns.boxplot(x='main.disorder', y='age', data=input)
plt.title('Age Distribution Across Different Disorders')
plt.xticks(rotation=90)
plt.savefig('content/Age Distribution Across Different Disorders.png', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='main.disorder', y='education', data=input)
plt.title('Education Level Distribution Across Different Disorders')
plt.xticks(rotation=90)
plt.savefig('content/Education Level Distribution Across Different Disorders.png', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='main.disorder', y='IQ', data=input)
plt.title('IQ Distribution Across Different Disorders')
plt.xticks(rotation=90)
plt.savefig('content/IQ Distribution Across Disorders.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(x='main.disorder', y='age', data=input)
plt.title('Age Distribution Across Different Disorders (Violin Plot)')
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(12, 6))
sns.violinplot(x='main.disorder', y='education', data=input)
plt.title('Education Level Distribution Across Different Disorders (Violin Plot)')
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(12, 6))
sns.violinplot(x='main.disorder', y='IQ', data=input)
plt.title('IQ Distribution Across Different Disorders (Violin Plot)')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Load your data (assuming 'input' is your DataFrame)
# input = pd.read_csv('your_data.csv')

# Extract features and target
# Assuming 'main.disorder' is the target variable and we are predicting a specific disorder, say 'Schizophrenia' (encoded as 5)
target_disorder = 10
input['target'] = (input['specific.disorder'] == target_disorder).astype(int)

# Select demographic and EEG features
demographic_features = ['age', 'education', 'IQ', 'sex']  # Add more demographic features if available
eeg_features = input.columns[7:1148]  # Assuming EEG features start from the 8th column

# Combine features
features = demographic_features + list(eeg_features)
X = input[features]
y = input['target']

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train logistic regression model
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

# Predict probabilities on the test set
y_probs = model.predict_proba(X_test)[:, 1]

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
auc = roc_auc_score(y_test, y_probs)

# Plot ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Print the AUC value
print(f'AUC: {auc:.2f}')


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Initialize GridSearchCV
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5, scoring='roc_auc')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_auc = grid_search.best_score_

print(f'Best Parameters: {best_params}')
print(f'Best AUC: {best_auc:.2f}')


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Fit the model
rf_model.fit(X_train, y_train)

# Predict probabilities
y_rf_probs = rf_model.predict_proba(X_test)[:, 1]

# Calculate ROC curve and AUC
rf_fpr, rf_tpr, rf_thresholds = roc_curve(y_test, y_rf_probs)
rf_auc = roc_auc_score(y_test, y_rf_probs)

# Plot ROC curve
plt.figure(figsize=(10, 6))
plt.plot(rf_fpr, rf_tpr, color='blue', label=f'Random Forest ROC curve (AUC = {rf_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Print the AUC value
print(f'Random Forest AUC: {rf_auc:.2f}')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

target_disorder = 5
input['target'] = (input['main.disorder'] == target_disorder).astype(int)

# Assuming 'input' is your DataFrame
# Demographic features
demographic_features = ['age', 'sex', 'education', 'IQ']

# EEG features
eeg_features = input.columns[20:1148]  # Adjust based on your column indices

# Select features
features = demographic_features + list(eeg_features)
x=input[features]
y = input['main.disorder']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Logistic Regression
logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train, y_train)
y_logreg_probs = logreg.predict_proba(X_test)

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_rf_probs = rf.predict_proba(X_test)

# Calculate ROC AUC for Logistic Regression using 'ovr' strategy
logreg_auc = roc_auc_score(y_test, y_logreg_probs, multi_class='ovr')
print(f'Logistic Regression AUC: {logreg_auc:.2f}')

# Calculate ROC AUC for Random Forest using 'ovr' strategy
rf_auc = roc_auc_score(y_test, y_rf_probs, multi_class='ovr')
print(f'Random Forest AUC: {rf_auc:.2f}')

# Compute ROC curve for each class and plot
plt.figure(figsize=(10, 6))

# Logistic Regression ROC curves
for i in range(len(logreg.classes_)):
    # Handle cases where predictions are 1D (single probability)
    if y_logreg_probs[:, i].ndim == 1:
        fpr, tpr, _ = roc_curve(y_test, y_logreg_probs[:, i], pos_label=logreg.classes_[i])
    else:
        fpr, tpr, _ = roc_curve(y_test, y_logreg_probs[:, i][:, 1], pos_label=logreg.classes_[i])  # Access second column if 2D

    #plt.plot(fpr, tpr, label=f'Logistic Regression class {logreg.classes_[i]} (AUC = {roc_auc_score(y_test, y_logreg_probs[:, i], multi_class="ovr"):.2f})')


# Random Forest ROC curves
for i in range(len(rf.classes_)):
    fpr, tpr, _ = roc_curve(y_test, y_rf_probs[:, i], pos_label=rf.classes_[i])
    #plt.plot(fpr, tpr, label=f'Random Forest class {rf.classes_[i]} (AUC = {roc_auc_score(y_test, y_rf_probs[:, i], multi_class="ovr"):.2f})')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()




In [ ]:
# Assuming `data` is your DataFrame
descriptive_stats = input.groupby('main.disorder')[['age', 'education', 'IQ', 'sex']].describe()
print(descriptive_stats)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# ANOVA for sex
model_IQ = ols('sex ~ C(Q("main.disorder"))', data=input).fit()
anova_table_IQ = sm.stats.anova_lm(model_IQ, typ=2)
print("ANOVA for sex:\n", anova_table_IQ)

# ANOVA for age
model_age = ols('age ~ C(Q("main.disorder"))', data=input).fit()
anova_table_age = sm.stats.anova_lm(model_age, typ=2)
print("ANOVA for Age:\n", anova_table_age)

# ANOVA for education
model_education = ols('education ~ C(Q("main.disorder"))', data=input).fit()
anova_table_education = sm.stats.anova_lm(model_education, typ=2)
print("ANOVA for Education:\n", anova_table_education)

# ANOVA for IQ
model_IQ = ols('IQ ~ C(Q("main.disorder"))', data=input).fit()
anova_table_IQ = sm.stats.anova_lm(model_IQ, typ=2)
print("ANOVA for IQ:\n", anova_table_IQ)

In [ ]:
# Correlation matrix
correlation_matrix = input[['age', 'education', 'IQ']].corr()
print("Correlation Matrix:\n", correlation_matrix)

In [ ]:
# Example: Creating a stacked bar plot using pandas
plt.style.use('default')  # Reset style to default
plt.rc('font', family='serif', size=16)

# Boxplot for age
plt.figure(figsize=(10, 4))
sns.boxplot(x='main.disorder', y='age', data=input)
plt.xlabel('Psychiatric Disorder',fontsize=20)
plt.ylabel('Age',fontsize=20)

# Add a grid with light dotted lines
plt.grid(True, linestyle=':', linewidth=0.75, color='black')
#plt.title('Age Distribution Across Disorders')
plt.setp(plt.gca().get_xticklabels(), fontsize=14, fontfamily='serif')
plt.setp(plt.gca().get_yticklabels(), fontsize=14, fontfamily='serif')
#plt.xticks(rotation=45, ha='right')
plt.xticks(ticks=[0, 1, 2, 3, 4, 5, 6], labels=['AddiD', 'AnxD', 'HC', 'MD', 'OCD', 'S', 'TSD'])
# Adjust layout to fit everything nicely
plt.tight_layout()
plt.savefig('content/Age Distribution Across Disorders.pdf', dpi=600, bbox_inches='tight')
plt.savefig('content/Age Distribution Across Disorders.png', dpi=600, bbox_inches='tight')
plt.show()

# Boxplot for education
plt.figure(figsize=(12, 6))
sns.boxplot(x='main.disorder', y='education', data=input)
plt.title('Education Distribution Across Disorders')
plt.xticks(rotation=45)
plt.show()

# Boxplot for IQ
plt.figure(figsize=(10, 4))
sns.boxplot(x='main.disorder', y='IQ', data=input)
plt.xlabel('Psychiatric Disorder',fontsize=20)
plt.ylabel('IQ',fontsize=20)
#plt.title('IQ Distribution Across Disorders')
# Add a grid with light dotted lines
plt.grid(True, linestyle=':', linewidth=0.75, color='black')
plt.setp(plt.gca().get_xticklabels(), fontsize=14, fontfamily='serif')
plt.setp(plt.gca().get_yticklabels(), fontsize=14, fontfamily='serif')
#plt.xticks(rotation=45, ha='right')
plt.xticks(ticks=[0, 1, 2, 3, 4, 5, 6], labels=['AddiD', 'AnxD', 'HC', 'MD', 'OCD', 'S', 'TSD'])
plt.tight_layout()
plt.savefig('content/IQ Distribution Across Disorders.pdf', dpi=600, bbox_inches='tight')
plt.savefig('content/IQ Distribution Across Disorders.png', dpi=600, bbox_inches='tight')
plt.show()

# Bar chart for sex
plt.figure(figsize=(10, 4))
sns.countplot(x='main.disorder', hue='sex', data=input)
plt.xlabel('Psychiatric Disorder', fontsize=20)
plt.ylabel('Count', fontsize=20)
plt.setp(plt.gca().get_xticklabels(), fontsize=14, fontfamily='serif')
plt.setp(plt.gca().get_yticklabels(), fontsize=14, fontfamily='serif')
legend = plt.legend(fontsize=17)
legend.get_texts()[0].set_text('Female')
legend.get_texts()[1].set_text('Male')
plt.xticks(ticks=[0, 1, 2, 3, 4, 5, 6], labels=['AddiD', 'AnxD', 'HC', 'MD', 'OCD', 'S', 'TSD'])
plt.tight_layout()
plt.savefig('content/Sex Distribution Across Disorder.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate percentages
data=input
data['percentage'] = data.groupby('main.disorder')['sex'].transform(lambda x: 100 * x.count() / len(data))

# Plot percentage bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x='main.disorder', y='percentage', hue='sex', data=data, estimator=lambda x: len(x) / len(data) * 100)
plt.title('Sex Distribution Across Disorders (Percentage)')
plt.xticks(rotation=45)
plt.ylabel('Percentage')
plt.show()

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Post-hoc test for sex
tukey_age = pairwise_tukeyhsd(endog=input['sex'], groups=input['main.disorder'], alpha=0.05)
print("Tukey HSD for Sex:\n", tukey_age)


# Post-hoc test for age
tukey_age = pairwise_tukeyhsd(endog=input['age'], groups=input['main.disorder'], alpha=0.05)
print("Tukey HSD for Age:\n", tukey_age)

# Post-hoc test for education
tukey_education = pairwise_tukeyhsd(endog=input['education'], groups=input['main.disorder'], alpha=0.05)
print("Tukey HSD for Education:\n", tukey_education)

# Post-hoc test for IQ
tukey_IQ = pairwise_tukeyhsd(endog=input['IQ'], groups=input['main.disorder'], alpha=0.05)
print("Tukey HSD for IQ:\n", tukey_IQ)
